In [1]:
import cv2
import numpy as np
import os
import csv
from scipy.signal import find_peaks, savgol_filter

In [2]:
def detect_steps(image):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    equalized = clahe.apply(blurred)

    edges = cv2.Canny(equalized, 70, 140)
    horizontal_profile = np.sum(edges, axis=1)

    window_size = max(11, int(image.shape[0] * 0.05) // 2 * 2 + 1)
    smoothed_profile = savgol_filter(horizontal_profile, window_length=window_size, polyorder=3)
    
    derivative = np.gradient(smoothed_profile)

    pos_peaks, _ = find_peaks(derivative, prominence=np.percentile(derivative, 75), distance=gray.shape[0]//20)
    neg_peaks, _ = find_peaks(-derivative, prominence=np.percentile(-derivative, 75), distance=gray.shape[0]//20)

    all_peaks = np.sort(np.concatenate([pos_peaks, neg_peaks]))
    valid_pairs = []
    
    for i in range(1, len(all_peaks)):
        prev_type = 'pos' if all_peaks[i-1] in pos_peaks else 'neg'
        curr_type = 'pos' if all_peaks[i] in pos_peaks else 'neg'
        
        # Une marche valide -> pic positif et negatif (peut etre faux)
        if prev_type == 'pos' and curr_type == 'neg':
            if (all_peaks[i] - all_peaks[i-1]) < gray.shape[0]/15:  # ecart maximal tolere
                valid_pairs.append((all_peaks[i-1], all_peaks[i]))
    
    return len(valid_pairs)

In [3]:
image_folder = 'images'
csv_file = 'data annotations - Feuille 1.csv'
output_csv = 'resultats.csv'

results = []
correct_predictions = 0
total_predictions = 0
total_error = 0

with open(csv_file, mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)

    # on ignore les 2 premieres ligens
    next(reader)
    next(reader)
    
    for row in reader:
        if len(row) < 2 or not row[0]:  # ignorer les lignes vides ou incompletes
            continue
        
        image_name = row[0]
        expected = int(row[1])
        image_path = os.path.join(image_folder, image_name)
        if not os.path.exists(image_path):
            #print(f"Erreur : fichier introuvable {image_name}, pas bien :(")
            continue
        
        # detection des marches
        image = cv2.imread(image_path)
        pred = detect_steps(image)

        error = abs(pred - expected)
        total_error += error
        is_correct = (error == 0)
        if is_correct:
            correct_predictions += 1
        total_predictions += 1
        results.append([image_name, expected, pred, is_correct, error])
        print(f"Image : {image_name} | attendues : {expected} | detectees : {pred} | Correct : {is_correct} | Erreur : {error} marches")

# calcule accuracy et de l'erreur
if total_predictions > 0:
    accuracy = correct_predictions/total_predictions
    mean_error = total_error/total_predictions
    print(f"Accuracy : {accuracy * 100:.2f}%")
    print(f"Erreur moyenne (MAE) : {mean_error:.2f} marches")
else:
    print("C nul")

# sauvegarde
with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Image', 'Marches attendues', 'Marches detectees', 'Correct', 'Erreur'])
    writer.writerows(results)
print(f"Resultats dans {output_csv}")


Image : Groupe2_Image1.jpeg | attendues : 4 | detectees : 11 | Correct : False | Erreur : 7 marches
Image : Groupe2_Image2.jpeg | attendues : 7 | detectees : 12 | Correct : False | Erreur : 5 marches
Image : Groupe2_Image3.jpeg | attendues : 9 | detectees : 10 | Correct : False | Erreur : 1 marches
Image : Groupe2_Image4.png | attendues : 6 | detectees : 13 | Correct : False | Erreur : 7 marches
Image : Groupe2_Image5.jpeg | attendues : 8 | detectees : 10 | Correct : False | Erreur : 2 marches
Image : Groupe2_Image6.jpeg | attendues : 10 | detectees : 7 | Correct : False | Erreur : 3 marches
Image : Groupe2_Image7.jpeg | attendues : 10 | detectees : 11 | Correct : False | Erreur : 1 marches
Image : Groupe2_Image11.jpg | attendues : 9 | detectees : 14 | Correct : False | Erreur : 5 marches
Image : Groupe2_Image12.png | attendues : 7 | detectees : 10 | Correct : False | Erreur : 3 marches
Image : Groupe2_Image13.png | attendues : 16 | detectees : 11 | Correct : False | Erreur : 5 marches